## Imports

In [2]:
LOCAL = True

In [3]:
if LOCAL:
  PATH = "./data"
else:
  PATH = "gdrive/MyDrive/application-project-abgabe"
  from google.colab import drive
  drive.mount('/content/gdrive')

In [ ]:
!pip install colab-env --upgrade


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install spotipy --upgrade


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install ratelimit


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install levenshtein


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import json
from numpy import NaN
from ratelimit import limits, sleep_and_retry
import Levenshtein
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel, polynomial_kernel, sigmoid_kernel, rbf_kernel, laplacian_kernel, chi2_kernel, euclidean_distances, manhattan_distances, cosine_distances
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm
import time
from ast import literal_eval

## Filterung der Datenbank um irrelevante Ergebnisse zu verhindern

In [70]:
#by_isrc = pd.read_csv('data/by_isrc.csv')
by_isrc = pd.read_csv(f"{PATH}/by_isrc.csv")

In [71]:
by_isrc["country_code"] = by_isrc["isrc"].apply(lambda x: x[:2])

In [72]:
by_isrc["country_code"].unique()

array(['AE', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BC', 'BE', 'BG', 'BR', 'BX',
       'Br', 'CA', 'CH', 'CI', 'CL', 'CN', 'CO', 'CS', 'CZ', 'DE', 'DG',
       'DK', 'EE', 'EG', 'ES', 'FI', 'FR', 'FX', 'GB', 'GN', 'GR', 'GT',
       'GX', 'HK', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IR', 'IS', 'IT',
       'JM', 'JP', 'KE', 'KR', 'KS', 'LB', 'LK', 'LT', 'LU', 'LV', 'MD',
       'MM', 'MT', 'MU', 'MX', 'MY', 'NG', 'NI', 'NL', 'NO', 'NZ', 'No',
       'PE', 'PH', 'PL', 'PT', 'QM', 'QZ', 'RE', 'RO', 'RS', 'RU', 'SE',
       'SG', 'SI', 'SK', 'SM', 'SW', 'TC', 'TH', 'TR', 'TW', 'UA', 'UK',
       'UR', 'US', 'UY', 'VE', 'VN', 'ZA', 'ZZ', 'ae', 'ca', 'gb', 'il',
       'lv', 'nl', 'qz', 'us', 've'], dtype=object)

Einige Ländercodes sind kleingeschrieben. Diese sollen in Großbuchstaben umgewandelt werden.

In [76]:
by_isrc["country_code"] = by_isrc["country_code"].str.upper()

In [77]:
by_isrc["country_code"].unique()

array(['AE', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BC', 'BE', 'BG', 'BR', 'BX',
       'CA', 'CH', 'CI', 'CL', 'CN', 'CO', 'CS', 'CZ', 'DE', 'DG', 'DK',
       'EE', 'EG', 'ES', 'FI', 'FR', 'FX', 'GB', 'GN', 'GR', 'GT', 'GX',
       'HK', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IR', 'IS', 'IT', 'JM',
       'JP', 'KE', 'KR', 'KS', 'LB', 'LK', 'LT', 'LU', 'LV', 'MD', 'MM',
       'MT', 'MU', 'MX', 'MY', 'NG', 'NI', 'NL', 'NO', 'NZ', 'PE', 'PH',
       'PL', 'PT', 'QM', 'QZ', 'RE', 'RO', 'RS', 'RU', 'SE', 'SG', 'SI',
       'SK', 'SM', 'SW', 'TC', 'TH', 'TR', 'TW', 'UA', 'UK', 'UR', 'US',
       'UY', 'VE', 'VN', 'ZA', 'ZZ'], dtype=object)

Gruppierung der Songs nach den Ländercodes.

In [78]:
countries = {
    'AE': 'United Arab Emirates',
    'AR': 'Argentina',
    'AT': 'Austria',
    'AU': 'Australia',
    'AZ': 'Azerbaijan',
    'BA': 'Bosnia and Herzegovina',
    'BC': 'Canada (British Columbia)',
    'BE': 'Belgium',
    'BG': 'Bulgaria',
    'BR': 'Brazil',
    'BX': 'Benelux',
    'CA': 'Canada',
    'CH': 'Switzerland',
    'CI': 'Ivory Coast (Côte d\'Ivoire)',
    'CL': 'Chile',
    'CN': 'China',
    'CO': 'Colombia',
    'CS': 'Serbia and Montenegro',
    'CZ': 'Czech Republic',
    'DE': 'Germany',
    'DG': 'Germany (East)',
    'DK': 'Denmark',
    'EE': 'Estonia',
    'EG': 'Egypt',
    'ES': 'Spain',
    'FI': 'Finland',
    'FR': 'France',
    'FX': 'France (Metropolitan)',
    'GB': 'United Kingdom',
    'GN': 'Guinea',
    'GR': 'Greece',
    'GT': 'Guatemala',
    'GX': 'Greece (Ancient)',
    'HK': 'Hong Kong',
    'HR': 'Croatia',
    'HU': 'Hungary',
    'ID': 'Indonesia',
    'IE': 'Ireland',
    'IL': 'Israel',
    'IN': 'India',
    'IR': 'Iran',
    'IS': 'Iceland',
    'IT': 'Italy',
    'JM': 'Jamaica',
    'JP': 'Japan',
    'KE': 'Kenya',
    'KR': 'South Korea',
    'KS': 'Kosovo',
    'LB': 'Lebanon',
    'LK': 'Sri Lanka',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'LV': 'Latvia',
    'MD': 'Moldova',
    'MM': 'Myanmar (Burma)',
    'MT': 'Malta',
    'MU': 'Mauritius',
    'MX': 'Mexico',
    'MY': 'Malaysia',
    'NG': 'Nigeria',
    'NI': 'Nicaragua',
    'NL': 'Netherlands',
    'NO': 'Norway',
    'NZ': 'New Zealand',
    'PE': 'Peru',
    'PH': 'Philippines',
    'PL': 'Poland',
    'PT': 'Portugal',
    'QM': 'Marshall Islands', #second country code for the United States
    'QZ': 'Unknown or Invalid Area',
    'RE': 'Réunion',
    'RO': 'Romania',
    'RS': 'Serbia',
    'RU': 'Russia',
    'SE': 'Sweden',
    'SG': 'Singapore',
    'SI': 'Slovenia',
    'SK': 'Slovakia',
    'SM': 'San Marino',
    'SW': 'Sweden',
    'TC': 'Turks and Caicos Islands',
    'TH': 'Thailand',
    'TR': 'Turkey',
    'TW': 'Taiwan',
    'UA': 'Ukraine',
    'UK': 'United Kingdom',
    'UR': 'Uruguay',
    'US': 'United States',
    'UY': 'Uruguay',
    'VE': 'Venezuela',
    'VN': 'Vietnam',
    'ZA': 'South Africa',
    'ZZ': 'Unknown or Invalid Area'
}

Erstellen eines neuen Features `country`, welches den Namen des jeweiligen Landes beinhaltet.

In [79]:
by_isrc["country"] = by_isrc["country_code"].apply(lambda x: countries[x])

Gruppieren der Daten nach `country` und `country_code`. Dies soll genutzt werden, um zu analysieren, aus welchem Land am meisten Songs veröffentlicht worden sind.

In [80]:
grouped_countries = by_isrc.copy()
grouped_countries = grouped_countries[["country_code", "country"]]
grouped_countries["num_countries"] = 0
grouped_countries = grouped_countries.groupby(["country_code", "country"]).count()

In [81]:
grouped_countries = grouped_countries.sort_values("num_countries", ascending=False)

In [82]:
grouped_countries.reset_index(inplace=True)

In [83]:
# country und country_code werden kombiniert für eine bessere Darstellung
grouped_countries["country_comb"] = grouped_countries.apply(lambda x: f'{x["country"]}-{x["country_code"]}', axis=1)

In [84]:
fig = px.bar(grouped_countries[:30], x="country_comb", y="num_countries", template="plotly_dark")
fig.update_layout(
    height=500,
    title="Veröffentlichte Songs pro Land (Top 30)",
    xaxis_title="Land",
    yaxis_title="Veröffentlichte Songs"
)
fig.show()

Es sollen alle Lieder entfernt werden welche nicht aus englischsprachigen Länger (US, UK, CA, AU, Neuseeland) sowie Deutschland, Schweiz, Österreich, Italien und Schweden stammen.

In [85]:
important_country_codes = ["US", "GB", "CA", "AU", "NZ", "DE", "CH", "AT", "IT", "SE"]

# kleinere Inselgegenden von Amerika: Baker Island, Howland Island, Jarvis Island, Johnston Atoll, Kingman Reef, Midway Islands, Navassa Island, Palmyra Atoll, Wake Island
minor_insular_areas_us = ["XB", "XH", "XQ", "XU", "XM", "QM", "XV", "XL", "QW"]
important_country_codes.extend(minor_insular_areas_us)

`QM` wird heute als zusätzlicher Country code für Amerika verwendet, da der Code `US` sein Limit erreicht hat. Da dieser Beschluss jedoch aus 2010 stammt, sollten die Lieder der 80er nicht beeinflusst werden?

In [86]:
by_isrc[by_isrc["country_code"] == "QM"].sort_values("popularity", ascending=False)[["isrc", "genres", "name", "artists", "popularity", "chart_power"]]

,isrc,genres,name,artists,popularity,chart_power
274154,QMKHM1600219,"['hard-rock', 'metal', 'rock']",master of puppets,metallica,80,NaN
274107,QMKHM1600096,"['hard-rock', 'metal', 'rock']",for whom the bell tolls - remastered,metallica,75,NaN
271243,QMFME1326440,"['latin', 'pop']",tu dama de hierro,marisela,71,NaN
274108,QMKHM1600097,"['hard-rock', 'metal', 'rock']",fade to black - remastered,metallica,70,NaN
274057,QMKHM1600034,"['hard-rock', 'metal', 'rock']",seek & destroy - remastered,metallica,69,NaN
...,...,...,...,...,...,...
267541,QM7281628419,['tango'],fuegos artificiales,juan d'arienzo y su orquesta típica,0,NaN
267551,QM7281713701,['jazz'],raincheck,nick brignola,0,NaN
267552,QM7281713702,['jazz'],tenderly,nick brignola,0,NaN
267553,QM7281713703,['jazz'],hurricane connie,nick brignola,0,NaN


Herausfiltern der Lieder aus den "wichtigen" Länder.

In [87]:
reduced_isrc = by_isrc.copy()
reduced_isrc = reduced_isrc[reduced_isrc["country_code"].isin(important_country_codes)]
reduced_isrc["genres"] = reduced_isrc["genres"].apply(literal_eval)

In [88]:
print(f"Die Anzahl der Lieder hat sich von {by_isrc.shape[0]} auf {reduced_isrc.shape[0]} verringert.")

Die Anzahl der Lieder hat sich von 416154 auf 251101 verringert.


Gruppieren der Daten nach den Genres => Welche Genres waren in den wichtigen Ländern relevant?

In [89]:
def filter_important_genres(df):
  important_genres = df.copy()
  important_genres = important_genres.explode("genres")
  important_genres["num"] = 0
  important_genres = important_genres[["genres", "num"]]
  important_genres = important_genres.groupby("genres").count()
  important_genres.reset_index(inplace=True)
  important_genres = important_genres.sort_values("num", ascending=False)
  return important_genres

In [90]:
filter_important_genres(reduced_isrc)

,genres,num
16,classical,63432
87,rock,60598
77,pop,48841
38,german,28985
62,jazz,22632
...,...,...
28,dubstep,2
12,cantopop,1
53,idm,1
69,metalcore,1


Am meisten Lieder wurden für das Genre Klassik veröffentlicht. Da dieses Genre nicht wirklich relevant ist, müssen zunächst die irrelevanten Genres entfernt werden. Hiebei gibt es zwei verschiedene Wege. Verfügt ein Künstler sowohl über ein erwünschtes als auch ein unerwünschtes Genre, so können entweder alle Songs von ihm behalten oder entfernt werden. Zunächst wird geprüft, wie häufig ein solcher Fall eintritt.

In [91]:
unwanted_genres = ["classical", "jazz", "folk", "french", "turkish", "gospel", "samba", "piano", "mpb", "sertanejo", "pagode", "sleep", "forro", "malay", "anime",
                 "j-idol", "comedy", "mandopop", "cantopop", "show-tunes", "emo", "romance", "j-dance", "chill", "world-music", "iranian", "idm", "metalcore",
                 "hardstyle", "opera", "k-pop", "j-pop", "j-rock", "happy"]

In [92]:
occ_unwanted_and_wanted = by_isrc.copy()
occ_unwanted_and_wanted = occ_unwanted_and_wanted[["genres"]]
occ_unwanted_and_wanted["genres"] = occ_unwanted_and_wanted["genres"].apply(literal_eval)
occ_unwanted_and_wanted = occ_unwanted_and_wanted.explode("genres")

In [93]:
wanted_genres = list(set(occ_unwanted_and_wanted["genres"].unique()).difference(unwanted_genres))

In [94]:
def filter_row(row):
    has_wanted_genre = any(genre in row['genres'] for genre in wanted_genres)
    has_not_wanted_genre = any(genre in row['genres'] for genre in unwanted_genres)
    return has_wanted_genre and has_not_wanted_genre

In [95]:
occ_unwanted_and_wanted_df = reduced_isrc[reduced_isrc.apply(filter_row, axis=1)]

In [96]:
print(f"Insgesamt gibt es {occ_unwanted_and_wanted_df.shape[0]} Lieder, welche über ein erwünschtes und ein unerwünschtes Genre verfügen.")

Insgesamt gibt es 48318 Lieder, welche über ein erwünschtes und ein unerwünschtes Genre verfügen.


In [97]:
print(f"Hierbei gibt es {len(list(occ_unwanted_and_wanted_df.genres.astype(str).unique()))} verschiedene Kombinationen der Genres.")

Hierbei gibt es 582 verschiedene Kombinationen der Genres.


## Visualisierung Kombination

Es werden alle Songs entfernt, welche mindestens ein unerwünschtes Feature beinhalten.

In [98]:
by_isrc_sub = by_isrc.copy()
by_isrc_sub = by_isrc_sub[['isrc', 'genres']]
by_isrc_sub = by_isrc_sub[~by_isrc_sub['genres'].str.contains('|'.join(unwanted_genres))]
by_isrc_sub['genres'] = by_isrc_sub['genres'].apply(literal_eval)

Das neu entstandene DataFrame soll nun mit dem bereits auf die wichtigen Länder reduzierten DataFrame gemerged werden.

In [215]:
#reduced_isrc.drop(columns="genres", inplace=True)

In [99]:
reduced_isrc.shape, by_isrc_sub.shape

((251101, 27), (220702, 2))

In [100]:
reduced_isrc = reduced_isrc.merge(by_isrc_sub[["isrc"]], how="inner", on="isrc")

In [101]:
print(f"Die Anzahl der Songs hat sich weiter auf {reduced_isrc.shape[0]} verringert. Die Anzahl der Features ist bei {reduced_isrc.shape[1]} geblieben.")

Die Anzahl der Songs hat sich weiter auf 139629 verringert. Die Anzahl der Features ist bei 27 geblieben.


In [102]:
important_genres = filter_important_genres(reduced_isrc)
important_genres = important_genres.sort_values("num", ascending=False)
important_genres.head(10)

,genres,num
61,rock,52805
51,pop,43525
56,punk,16552
14,country,12353
47,metal,11743
35,hard-rock,9228
29,german,8980
20,disco,8065
70,swedish,7964
68,soul,7289


In [103]:
fig = px.bar(important_genres[:30], x="genres", y="num", template="plotly_dark")
fig.update_layout(
    height=500,
    title="Veröffentlichte Songs pro Genre in den wichtigen Ländern (Top 30)",
    xaxis_title="Genre",
    yaxis_title="Veröffentlichte Songs"
)
fig.show()

In [224]:
#reduced_isrc.to_csv(f"{PATH}/reduced_isrc.csv")

# Auffindbarkeiten

## Experiment

In [5]:
by_isrc = pd.read_csv(f"{PATH}/reduced_isrc.csv")

In [6]:
@sleep_and_retry
@limits(calls=60, period=30)
def call_api():
  return

In [7]:
chart_power_df = pd.read_excel(f"{PATH}/chart-power-scores_80s.xlsx")
chart_power_df = chart_power_df.applymap(lambda s: s.lower() if type(s) == str else s)
chart_power_df = chart_power_df[['Song', 'Artist', 'Points']].groupby(['Song', 'Artist']).sum()
chart_power_df.reset_index(inplace=True)

In [8]:
if LOCAL:
    from dotenv import load_dotenv
    load_dotenv()
    SPOTIFY_CLIENT_ID = os.environ.get("SPOTIFY_CLIENT_ID")
    SPOTIFY_CLIENT_SECRET = os.environ.get("SPOTIFY_CLIENT_SECRET")
else:
    import colab_env
    SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
    SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

In [9]:
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Abfrage aller Lieder für Boney M.

Boney M. ist einer der Künstler, welche in der erstellen Datenbank nicht vorkommen. Dieses Problem soll hier genauer untersucht werden. Hierfür sollen zunächst alle Lieder von Bony M. ohne weitere Filter abgefragt werden.

In [10]:
def query(year: int, offset: int):
  res = sp.search(q=f"artist:Boney M. year:{year}", type="track", limit=10, offset=offset, market="DE")
  if (len(res["tracks"]["items"]) == 10):
    return query(year, offset + 10)
  return len(res["tracks"]["items"]) + offset

In [12]:
boney_songs = pd.DataFrame()
for year in range(1980, 1990):
  boney_songs = pd.concat([boney_songs, pd.DataFrame([[year, query(year, 0)]], columns=["year", "songs"])])

In [ ]:
fig = px.bar(boney_songs, x="year", y="songs", title="Songs veröffentlich von Boney M. in den 80er Jahren")
fig.update_layout(
    height=500,
    template='plotly_dark',
    xaxis_title="Jahr",
    yaxis_title="Veröffentlichte Songs"
)
fig.show()

Wie in der oberen Abbildung sichtbar wird, wurden sehr wohl Lieder für Boney M. veröffentlicht. Einige davon auch in der 80er Jahren.

#### Einbauen des Genres in die query

Im nächsten Schritt sollen die Lieder für Boney M. mit den Filtern year und genre erfragt werden.

In [111]:
def genre_query(artist: str, year: int, offset:int, genres):
  for genre in genres:
    call_api()
    res = sp.search(q=f"artist:{artist} year:{year} genre:{genre}", type="track", limit=50, offset=offset, market="DE")
    if (len(res["tracks"]["items"] ) > 1):
      print(year, genre)
      #print(res)

In [112]:
def test_genre_query(artist: str):
  genres = sp.recommendation_genre_seeds()['genres']
  for year in range(1980, 1990):
    print(year)
    genre_query(artist, year, 0, genres)

In [114]:
test_genre_query("Boney M.")

1980
1981
1982
1983
1984
1985
1986
1987
1988
1989


Es existieren keine Songs für Boney M. wenn ein Genre in der Abfrage spezifiziert wird. Eventuell sind die Genres das Problem für die fehlenden Daten.

#### Abfrage nach dem Künstler Elton John.

Dasselbe soll auch für Elton John geschehen. Dies ist ein Künstler der in der Datenbank vorkommt und somit auch ein Genre zugewiesen bekommen sollte.

In [115]:
test_genre_query("Elton John")

1980
1980 piano
1980 rock
1981
1981 piano
1981 rock
1982
1982 piano
1982 rock
1983
1983 piano
1983 rock
1984
1984 piano
1984 rock
1985
1985 piano
1985 rock
1986
1986 piano
1986 rock
1987
1988
1988 piano
1988 rock
1989
1989 piano
1989 rock


Für Elton John existieren Genres. Im nächsten Schritt soll überprüft werden, ob die Genres auch in dem Response der Spotify API wiedergefunden wird.

In [ ]:
res = sp.search(q="artist:Elton John", type="track", market="DE", limit=50, offset=0)
res = res["tracks"]
if "items" in res:
  for item in res["items"]:
    if "artists" in item:
      #print(item["artists"]["genres"] if "genres" in item["artists"] else print("No genres available!"))
      #print(item["artists"])
      for artist in item["artists"]:
        print(artist["genres"] if "genres" in artist else artist.keys())
    else:
      print("No artists in item")
else:
  print("No items in res")
  print(res.keys())


dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])
dict_keys(

Die Spotify gibt auch für Künstler die ein Genre zugewiesen bekommen haben kein Genre zurück. Laut API sollte es ein weiteres Attribut `genres` geben.

Aufgrund dieser Ergebnisse werden bei den folgenden Abfragen der Lieder aus den Charts die Genres zunächst vernachlässigt.

#### Preprocessing

Wenn mehrere Künstler zusammen einen Song veröffentlich haben, sind diese hier mit einem `&` getrennt. Dies muss zunächst aufgelöst werden.

In [13]:
chart_power_df["Artist"] = chart_power_df["Artist"].str.split(" & ")
chart_power_df_explode = chart_power_df.explode("Artist", ignore_index=True)

In [14]:
chart_power_df_explode

,Song,Artist,Points
0,19,paul hardcastle,1600
1,'cause you're young,c. c. catch,998
2,(carmen) danger in her eyes,deborah sasson,531
3,(carmen) danger in her eyes,mcl,531
4,(do) the hucklebuck,coast to coast,697
...,...,...,...
2022,zu spät,die ã„rzte,605
2023,zuppa romana,schrott nach 8,601
2024,â€¦ und ganz doll mich (ich mag),rolf und seine freunde,1314
2025,über sieben brücken mußt du geh'n,karat,826


Einige Künstler und Lieder enthalten zudem Umlaute, welche zunächst entsprechend formatiert werden müssen (z.B. `Die Ärzte` sind als `Die ã„rzte` gespeichert).

In [15]:
replace = {
    "ã¶": "ö",
    "ã„": "ä",
    "ã–": "ö",
    "ã©": "e"
}

chart_power_df_explode["Song"] = chart_power_df_explode["Song"].replace(replace, regex=True)
chart_power_df_explode["Artist"] = chart_power_df_explode["Artist"].replace(replace, regex=True)

In [16]:
chart_power_df_explode[chart_power_df_explode["Song"] == "zu spät"]

,Song,Artist,Points
2022,zu spät,die ärzte,605


In [17]:
chart_power_df_explode[chart_power_df_explode["Artist"] == "die ã„rzte"]

,Song,Artist,Points


In [18]:
chart_power_df_explode[chart_power_df_explode["Artist"].str.contains("\[")]

,Song,Artist,Points
64,all of me (boy oh boy),sabrina [it],841
85,amoureux solitaires,lio [be],1555
100,another life,kano [it],1460
187,boys (summertime love),sabrina [it],1444
198,bridge to your heart,wax [uk],614
256,change your mind,raff [it],736
314,cry softly,secret service [se],451
351,delirio mind,scotch [it],1120
390,disco band,scotch [it],1806
480,ein weißes blatt'l papier,relax [de],535


Weiterhin ist hinter einigen Künstlern das Herukunftsland in eckigen Klammern gekennzeichnet. Dies muss ebenfalls entfernet werden, da diese andernfalls nicht von der Spotify API gefunden werden.

In [19]:
pattern = r' \[.*?\]'
chart_power_df_explode["Artist"] = chart_power_df_explode["Artist"].str.replace(pattern, '',regex=True)

In [20]:
chart_power_df_explode[chart_power_df_explode["Artist"].str.contains("\[")]

,Song,Artist,Points


In [21]:
len(chart_power_df_explode["Artist"].unique())

944

#### Funktionen zur Datenabfrage

In [22]:
def sp_request(req, retries, max_retries, *args):
  '''
  Wrapper für Requests an die Spotify API. Stellt sicher, dass nicht zu viele Requests auf einmal ausgeführt werden und dann Requests wiederholt werden, sollten sie fehlschlagen.

  Parameter
  ---------
  req: Function
    Funktion die ausgeführt werden soll

  retries: int
    Anzahl des derzeitigen Versuches den Request durchzuführen. Sollte in der Regel 0 betragen.

  max_retries: int
    Maximale Versuche die vollführt werden sollen, bevor die Funktion fehlschlägt.

  *args: 
    Variable Anzahl an Paramtern wecher der in req definierten Funktion übergeben werden sollen

  Return
  ------
  Das Ergebnis der req Funktion.
  '''
  try:
    call_api()
    return req(*args)
  except Exception as e:
    if isinstance(e, spotipy.exceptions.SpotifyException) and e.http_status == 429:
      # Handle rate limiting (HTTP 429)
      if retries < max_retries:
          sleep_time = exponential_backoff(retries)
          print(f"Rate limited, waiting for {sleep_time} seconds before retrying...")
          time.sleep(sleep_time)
          retries += 1
          return sp_request(req, retries, max_retries, *args)
      else:
          print("Max retries reached, exiting...")
    else:
      print(f"An error occurred: {e}")
      retries += 1
      return sp_request(req, retries, max_retries, *args)

# Define a function for exponential backoff
def exponential_backoff(retries):
    return 2 ** retries

In [23]:
def filter_track_features(track, features, genre, chart_power=False):
    '''
    Filters the relevant features of a track in returns them in JSON object.

    Parameter
    ---------
    track: Object
        Track returend by the spotify API

    features: Object
        Features of the object

    genre: string
        Genre that should be used

    chart_power: string | boolean; default=False
        Chart power for the song. False if no Chart power is available

    Return
    ------
    relevant_features: Object
        JSON Object that contains the relevant featues
    '''

    external_ids = track['external_ids'] if 'external_ids' in track else {}
    isrc = external_ids['isrc'] if 'isrc' in external_ids else np.nan
    artist_names = []

    if 'artists' in track and isinstance(track['artists'],list):
        for artist in track['artists']:
            if 'name' in artist:
                artist_names.append(artist['name'])
        artist_names = ','.join(artist_names)
    else:
        artist_names = np.nan



    if 'album' in track:
        album = track['album']['name'] if 'name' in track['album'] else np.nan
        release_date = track['album']['release_date'] if 'release_date' in track['album'] else np.nan
        release_date_precision = track['album']['release_date_precision'] if 'release_date_precision' in track['album'] else np.nan
    else:
         album = np.nan
         release_date = np.nan
         release_date_precision = np.nan


    track_name = track['name'] if 'name' in track else np.nan
    if chart_power == False:
        if track_name != np.nan and artist_names != np.nan:
            points = chart_power_df_explode.loc[(chart_power_df_explode.Song == track_name.lower()) & (chart_power_df_explode.Artist.str.contains(artist_names.lower()))]['Points']
            if points.empty:
                points = np.nan
            else:
                try:
                    points = int(points)
                except Exception as e:
                    print(e)
                    print(track_name)
                    print(artist_names)
                    print(points)
                    print("-------")
        else:
            points = np.nan
    else:
        points = chart_power

    return {
        'name': track_name,
        'artists': artist_names,
        'album': album,
        'release_date': release_date,
        'release_date_precision': release_date_precision,
        'spotify_id': track['id'] if 'id' in track else np.nan,
        'chart_power': points,
        'uri': track['uri'] if 'uri' in track else np.nan,
        'popularity': track['popularity'] if 'popularity' in track else np.nan,
        'genres': np.nan if genre == False else genre,
        'danceability': features['danceability'] if 'danceability' in features else np.nan,
        'energy': features['energy'] if 'energy' in features else np.nan,
        'key': features['key'] if 'key' in features else np.nan,
        'loudness': features['loudness'] if 'loudness' in features else np.nan,
        'mode': features['mode'] if 'mode' in features else np.nan,
        'speechiness': features['speechiness'] if 'speechiness' in features else np.nan,
        'acousticness': features['acousticness'] if 'acousticness' in features else np.nan,
        'instrumentalness': features['instrumentalness'] if 'instrumentalness' in features else np.nan,
        'liveness': features['liveness'] if 'liveness' in features else np.nan,
        'valence': features['valence'] if 'valence' in features else np.nan,
        'tempo': features['tempo'] if 'tempo' in features else np.nan,
        'duration_ms': features['duration_ms'] if 'duration_ms' in features else np.nan,
        'time_signature': features['time_signature'] if 'time_signature' in features else np.nan,
        'isrc': isrc,
    }

In [24]:
def find_closest_string(target, string_list):
  '''
  Durchsucht eine Liste an Strings und filtert das änhlichte Item heraus.

  Parameter
  ---------
  target: str
    Referenzinstanz 
  
  string_list: list
    Liste welche durchsucht werden soll

  
  Return
  ------
  closest_string: str
    Das ähnlichste Item

  min_distance: float | int
    Die Distanz vom closest_string zu target (nach Levenshtein)

  closest_index: int
    Index von closest_string in string_list
  '''
  try:
    closest_string = None
    min_distance = float('inf')
    closest_index = -1

    for index, string in enumerate(string_list):
        distance = Levenshtein.distance(target, string)
        if distance < min_distance:
            min_distance = distance
            closest_string = string
            closest_index = index

    return closest_string, min_distance, closest_index
  except Exception as e:
    print(target)
    print(string_list)
    print(e)

### Datenabfrage

In [104]:
found = 0

artist_over_1000 = []
no_tracks = []
no_features = []
artists_not_found = []

df_found = pd.DataFrame()
cols = ["Song", "Artist", "Points"]
df_missing = pd.DataFrame(columns=cols)
with tqdm(total=len(chart_power_df_explode)) as progress_bar:
    tracks_ = []
    audio_features_ = []
    points_ = []

    for index, item in chart_power_df_explode.iterrows():
        song = item["Song"]
        artist = item["Artist"]
        points = item["Points"]
        res = sp_request(lambda song, artist: sp.search(q=f"artist:{artist} track:{song}", market="DE", type="track"), 0, 100, song, artist)
        if "tracks" not in res:
            print("No tracks in result")
            df_missing = pd.concat([df_missing, pd.DataFrame([[song, artist, points]], columns=cols)], ignore_index=True)
            progress_bar.update(1)
            continue
        
        tracks = res["tracks"]

        ts = []
        if "items" in tracks and isinstance(tracks["items"], list):
            for track in tracks["items"]:
                ts.append(track)
        else:
            df_missing = pd.concat([df_missing, pd.DataFrame([[song, artist, points]], columns=cols)], ignore_index=True)
            progress_bar.update(1)
            continue

        if len(ts) == 0:
            df_missing = pd.concat([df_missing, pd.DataFrame([[song, artist, points]], columns=cols)], ignore_index=True)
            progress_bar.update(1)
            continue
        try:
            closest, distance, index = find_closest_string(str(song), [str(track["name"]) for track in ts])
        except Exception as e:
            print(e)
            continue
        tracks_.append(ts[index])
        points_.append(points)

        ts = []
        if len(tracks_) == 100:
            audio_features_ = sp_request(lambda x: sp.audio_features([track["id"] for track in x]), 0, 100, tracks_)
            track_features = []
            if len(tracks_) != len(audio_features_):
                print("Tracks and features have not the same lengh!")
                print(len(tracks_), len(audio_features_))
            for t, af, p in list(zip(tracks_, audio_features_, points_)):
                if not t:
                    print("No track!")
                    print(t)
                    print(af)
                    no_tracks.append(af)
                    continue
                if not af:
                    print("No features")
                    print(af)
                    print(t)
                    no_features.append(t)
                    continue
                features = filter_track_features(t, af, False, p)
                track_features.append(features)
                found = found + 1
            df_found = pd.concat([df_found, pd.DataFrame(track_features)], ignore_index=True)
            tracks_ = []
            audio_features_ = []
            points_ = []
        progress_bar.update(1)

    audio_features_ = sp_request(lambda x: sp.audio_features([track["id"] for track in x]), 0, 100, tracks_)
    track_features = []
    if len(tracks_) != len(audio_features_):
        print("Tracks and features have not the same lengh!")
        print(len(tracks_), len(audio_features_))
    for t, af, p in list(zip(tracks_, audio_features_, points_)):
        if not t:
            print("No track!")
            print(t)
            print(af)
            no_tracks.append(af)
            continue
        if not af:
            print("No features")
            print(af)
            print(t)
            no_features.append(t)
            continue
        features = filter_track_features(t, af, False, p)
        track_features.append(features)
        found = found + 1
    df_found = pd.concat([df_found, pd.DataFrame(track_features)], ignore_index=True)
    tracks_ = []
    audio_features_ = []


print(f"Von {len(chart_power_df_explode)} Songs wurden {found} gefunden.")

  0%|          | 0/2027 [00:00<?, ?it/s]

An error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


100%|██████████| 2027/2027 [17:01<00:00,  1.99it/s]  

Von 2027 Songs wurden 1729 gefunden.


In [79]:
df_found.isna().any()

name                      False
artists                   False
album                     False
release_date              False
release_date_precision    False
spotify_id                False
chart_power               False
uri                       False
popularity                False
genres                     True
danceability              False
energy                    False
key                       False
loudness                  False
mode                      False
speechiness               False
acousticness              False
instrumentalness          False
liveness                  False
valence                   False
tempo                     False
duration_ms               False
time_signature            False
isrc                      False
dtype: bool

In [106]:
# df_found.to_csv(f"{PATH}/chart-power-spotify.csv")
# df_missing.to_csv(f"{PATH}/chart-power-spotify-missing.csv")

In [27]:
df_found = pd.read_csv(f"{PATH}/chart-power-spotify.csv")
df_missing = pd.read_csv(f"{PATH}/chart-power-spotify-missing.csv")

### Visualisierung

- Sortierung der Chartpower Songs nach Chartscore und Zuweisung eines Perzentils zu jedem Song
- Sortierung der Chartpower Songs nach Spotify Popularity Score und Zuweisung eines Perzentils zu jedem Song
- Visualisierung der Perzentile im Scatterplot (Plotly)

In [107]:
bins = [-1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ["0 - 10", "11 - 20", "21 - 30", "31 - 40", "41 - 50", "51 - 60", "61 - 70", "71 - 80", "81 - 90", "91 - 100"]

def create_percentiles(df_):
    df_ = df_.sort_values(by="chart_power")
    df_['chart_power_percentile'] = (df_['chart_power'].rank(pct=True) * 100)
    # df_['chart_power_percentile'] = pd.cut(df_['chart_power_percentile'], bins=bins, labels=labels)

    df_ = df_.sort_values(by="popularity")
    df_["popularity_percentile"] = (df_["popularity"].rank(pct=True) * 100)
    # df_["popularity_percentile"] = pd.cut(df_["popularity_percentile"], bins=bins, labels=labels)
    return df_

In [108]:
df_found_copy = df_found.copy()
df_found_copy = create_percentiles(df_found_copy)

In [109]:
df_found_copy.head(1)

,name,artists,album,release_date,release_date_precision,spotify_id,chart_power,uri,popularity,genres,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,isrc,chart_power_percentile,popularity_percentile
1527,Tri tra trullala (Herbergsvater),Joachim Witt,Ballermann - German Party Songs,2023-09-08,day,5qrQxVQVwT03Iw7CpPvKsh,400,spotify:track:5qrQxVQVwT03Iw7CpPvKsh,0,NaN,...,0.0226,0.00899,0.096,0.834,131.255,285600,4,DEA629660020,0.057837,1.069983


In [110]:
fig = px.scatter(df_found_copy, x="popularity_percentile", y="chart_power_percentile", hover_data=["name", "artists"])
fig.update_yaxes(categoryorder="array", categoryarray=labels)
fig.update_layout(
    title="Popularity vs Chart Power",
    template="plotly_dark",
    xaxis_title="Popularity",
    yaxis_title="Chart Power"
)
fig.show()

In [42]:
def create_binned_scatter(df_original, title="Popularity vs Chart Power"):
    df_ = df_original.copy()
    df_ = df_[["popularity_percentile", "chart_power_percentile"]]
    df_["count"] = 0
    df_ = df_.groupby(["popularity_percentile", "chart_power_percentile"]).count()
    df_.reset_index(inplace=True)

    fig = px.scatter(df_, x="popularity_percentile", y="chart_power_percentile", size="count")
    fig.update_yaxes(categoryorder="array", categoryarray=labels)
    fig.update_layout(
        title=title,
        template="plotly_dark",
        xaxis_title="Popularity",
        yaxis_title="Chart Power"
    )
    fig.show()


create_binned_scatter(df_found_copy)

In [43]:
def create_heatmap(df_, title="Popularity vs Chart Power"):
    fig = px.density_heatmap(df_, x="popularity_percentile", y="chart_power_percentile", nbinsx=10, nbinsy=10, color_continuous_scale="Viridis")
    fig.update_yaxes(categoryorder="array", categoryarray=labels)
    fig.update_layout(
        title=title,
        template="plotly_dark",
        xaxis_title="Popularity",
        yaxis_title="Chart Power"
    )
    fig.show()

create_heatmap(df_found_copy)

### Vergleichbarkeit

- Vergleich: Deutscher Produktionen an Chartpower Songs vs. Deutsche Produktionen an Spotify Popularity
- Zusammensetzung der Populärsten Songs nach Herkunftsländern (Chartpower vs. Spotify Popularity)


In [33]:
df_comparison = df_found.copy()

In [34]:
df_comparison["country_code"] = df_comparison["isrc"].apply(lambda x: x[:2])
df_comparison["country_code"] = df_comparison["country_code"].str.upper()
df_comparison = df_comparison[df_comparison["country_code"] == "DE"]

In [35]:
df_comparison = create_percentiles(df_comparison)

In [44]:
create_heatmap(df_comparison, "Popularity vs Chart Power - DE")

In [45]:
create_heatmap(df_found_copy, "Popularity vs Chart Power - Weltweit")

In [46]:
create_binned_scatter(df_comparison, "Popularity vs Chart Power - DE")

In [47]:
create_binned_scatter(df_found_copy, "Popularity vs Chart Power - Weltweit")